## 提前打开的页面
### 各种卷积动图
https://github.com/vdumoulin/conv_arithmetic/
### LeNet 
http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf
### PPT
PPT-01-05 P.15

PPT-01-06

## Assignments Review
童鞋们的作业整体上非常好，讲过的内容中基本没什么问题；满分的童鞋比例非常高，

PPT.3 讲完后，暂不翻页

>互动-有奖竞猜

Q：为啥 loss 会这么大

A：scale

PPT.4

上节课说过了，再重复下，建议大家多尝试下不同的 scale

## Agenda
如果有时间的话，会带着大家一起，使用 Gluon 实现 MLP

### PPT.5 FCN
这个是我们前5次课一直在用的网络结构，这样的网络结构叫全连接，为什么叫全连接呢？因为是全部连接

### 跳转到 PPT-01-05 P.15-16
在两层网络中，输入层的每个单元与输出层的每个单元都相联

### PPT-01-05 P.17
在三层网络（如MLP）中，输入层的每个单元与隐层的每个单元都相联，隐层的每个单元都相联，与输出层的每个单元都相联


同层不相连，跨层不相连

### 回到 PPT-01-06 P.5

全连接神经网络之所以不太适合图像识别任务，主要有以下几个方面的问题：

- 参数数量太多 考虑一个输入1000*1000像素的图片(一百万像素，现在已经不能算大图了)，输入层有1000*1000=100万节点。假设第一个隐藏层有100个节点(这个数量并不多)，那么仅这一层就有(1000*1000+1)*100=1亿参数，这实在是太多了！我们看到图像只扩大一点，参数数量就会多很多，因此它的扩展性很差。
- 没有利用像素之间的位置信息 对于图像识别任务来说，每个像素和其周围像素的联系是比较紧密的，和离得很远的像素的联系可能就很小了。如果一个神经元和上一层所有神经元相连，那么就相当于对于一个像素来说，把图像的所有像素都等同看待，这不符合前面的假设。当我们完成每个连接权重的学习之后，最终可能会发现，有大量的权重，它们的值都是很小的(也就是这些连接其实无关紧要)。努力学习大量并不重要的权重，这样的学习必将是非常低效的。
- 网络层数限制 我们知道网络层数越多其表达能力越强，但是通过梯度下降方法训练深度全连接神经网络很困难，因为全连接神经网络的梯度很难传递超过3层。因此，我们不可能得到一个很深的全连接神经网络，也就限制了它的能力。

## PPT.6 初识CNN
有问题就要解决问题，于是CNN就应运而生了，先给个定义，初步认识下 CNN

那么，卷积神经网络又是怎样解决这个问题的呢？主要有三个思路：

- 局部连接 这个是最容易想到的，每个神经元不再和上一层的所有神经元相连，而只和一小部分神经元相连。这样就减少了很多参数。
- 权值共享 一组连接可以共享同一个权重，而不是每个连接有一个不同的权重，这样又减少了很多参数。
- 下采样 可以使用Pooling来减少每层的样本数，进一步减少参数数量，同时还可以提升模型的鲁棒性。
对于图像识别任务来说，卷积神经网络通过尽可能保留重要的参数，去掉大量不重要的参数，来达到更好的学习效果。



我们之前最多的层也只有3层，但是在CNN中层数一下就多起来了，比如最经典的 LeNet-5 共有7层，大家先简单的记下网络结构是卷积、pooling、卷积、pooling、全连接

接下来，我们将详细讨论这三个方面

## PPT.7-8 局部连接/部分连接


## PPT.9 2D VS. 1D
在FCN中，我们会将 image 拉伸成1维数组，比如 $28*28$ 的一张图，会拉伸成 $1*784$ 的一维数组


但是我们刚刚讲过了，这样会带来各种问题；因此在 CNN 中，通过卷积核，提取二维或更高维上的特征

至此，我们对卷积神经网络有了最基本的感性认识。接下来，我们将介绍卷积神经网络中各种层的计算和训练。

### 最简体验-卷积核 1*2

PPT.5

[5 6] 简单的理解，卷积核就是滑动窗口，有计算机网络基础，图像基础的童鞋对这个概念应该很熟，没有学过相关课程的童鞋也不用担心，非常容易理解。[5 6] 作为卷积核在下面的这个数据上滑动

[1 2 7 8]



$5*1+6*2 = 5 + 12 = 17$

$5*2+6*7 = 10 + 42 = 52$

$5*7+6*8 = 35 + 48 = 83$

切换到 Jupyter

In [ ]:
from mxnet import nd

In [13]:
x1 = nd.array([[1, 2],[2, 7],[7, 8]])
w1 = nd.array([5, 6])

In [14]:
nd.dot(x1,w1)


[ 17.  52.  83.]
<NDArray 3 @cpu(0)>

In [15]:
x2 = nd.array([1, 2, 7, 8]).reshape((1,1,1,4))
w2 = nd.array([5, 6]).reshape((1,1,1,2))
b = nd.array([0])

In [16]:
nd.Convolution(x2, w2, b, kernel=w.shape[2:], num_filter=w.shape[1])


[[[[ 17.  52.  83.]]]]
<NDArray 1x1x1x3 @cpu(0)>

稍微解释下 x 和 w 的 shape，每个 shape 分为两个部分，x（即 data）前两个是 batch、channel；输入为什么会有多个channel呢？因为一张图是rgb三个channel；w 前两个是 output channels 和 input channels；后面会进一步解释，目前大家简单理解为 MXNet NDArray 的套路即可；后半部分我们现在就要搞懂，都是 height * width，也是我们一直所说的 m 行 n 列。

我们第一个示例是在一个 $1*4$ 的数据上，以一个 $1*2$ 的核做卷积

In [19]:
x3 = nd.array([1, 2, 7, 8, 9, 10]).reshape((1,1,1,6))
w3 = nd.array([5, 6]).reshape((1,1,1,2))
b = nd.array([0])

In [20]:
nd.Convolution(x3, w3, b, kernel=w.shape[2:], num_filter=w.shape[1])


[[[[  17.   52.   83.   94.  105.]]]]
<NDArray 1x1x1x5 @cpu(0)>

In [21]:
nd.Convolution(x3, w2, b, kernel=w.shape[2:], num_filter=w.shape[1], stride=(2,2))


[[[[  17.   83.  105.]]]]
<NDArray 1x1x1x3 @cpu(0)>

切换到 PPT.6

### 卷积核 3*3

我们第二个示例是在一个 $5*5$ 的数据上，以一个 $3*3$ 的核做卷积

\begin{align}
a_{0,0}&=f(\sum_{m=0}^{2}\sum_{n=0}^{2}w_{m,n}x_{m+0,n+0}+w_b)\\
&=relu(w_{0,0}x_{0,0}+w_{0,1}x_{0,1}+w_{0,2}x_{0,2}+w_{1,0}x_{1,0}+w_{1,1}x_{1,1}\\
&+w_{1,2}x_{1,2}+w_{2,0}x_{2,0}+w_{2,1}x_{2,1}+w_{2,2}x_{2,2}+w_b)\\
&=relu(1+0+1+0+1+0+0+0+1+0)\\
&=relu(4)\\
&=4
\end{align}

10个数，讲完 PPT.9 切换到 Jupyter

In [21]:
x2 = nd.array([[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[0,0,1,1,0],[0,1,1,0,0]]).reshape((1,1,5,5))
x2


[[[[ 1.  1.  1.  0.  0.]
   [ 0.  1.  1.  1.  0.]
   [ 0.  0.  1.  1.  1.]
   [ 0.  0.  1.  1.  0.]
   [ 0.  1.  1.  0.  0.]]]]
<NDArray 1x1x5x5 @cpu(0)>

In [22]:
w2 = nd.array([[1,0,1],[0,1,0],[1,0,1]]).reshape((1,1,3,3))
w2


[[[[ 1.  0.  1.]
   [ 0.  1.  0.]
   [ 1.  0.  1.]]]]
<NDArray 1x1x3x3 @cpu(0)>

In [23]:
b2 = nd.array([0])

In [25]:
yhat = nd.Convolution(x2, w2, b2, kernel=w2.shape[2:], num_filter=w.shape[1])
yhat


[[[[ 4.  3.  4.]
   [ 2.  4.  3.]
   [ 2.  3.  4.]]]]
<NDArray 1x1x3x3 @cpu(0)>

In [26]:
yhat = nd.Convolution(x2, w2, b2, kernel=w2.shape[2:], num_filter=w.shape[1], stride=(2,2))
yhat


[[[[ 4.  4.]
   [ 2.  4.]]]]
<NDArray 1x1x2x2 @cpu(0)>

### 多个 channel

In [5]:
x = nd.arange(18).reshape((1,2,3,3))
w = nd.arange(8).reshape((1,2,2,2))
b = nd.array([1])
yhat = nd.Convolution(data, w, b, kernel=w.shape[2:], num_filter=w.shape[0])

print('input:', x, '\n\nweight:', w, '\n\nbias:', b, '\n\noutput:', yhat)

input: 
[[[[  0.   1.   2.]
   [  3.   4.   5.]
   [  6.   7.   8.]]

  [[  9.  10.  11.]
   [ 12.  13.  14.]
   [ 15.  16.  17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 

weight: 
[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 4.  5.]
   [ 6.  7.]]]]
<NDArray 1x2x2x2 @cpu(0)> 

bias: 
[ 1.]
<NDArray 1 @cpu(0)> 

output: 
[[[[ 269.  297.]
   [ 353.  381.]]]]
<NDArray 1x1x2x2 @cpu(0)>


input 1x2x3x3，1先不管，2x3x3 表示是2个 channel，2个2D Array; weight要与之对应，因此也是2个 channel；具体运算的时候，是weight的第一个channel与input第一个channel做卷积；weight的第2个channel与input第2个channel做卷积；运算结果为2个output；最后将这两个output再加起来，得到最终的output

# 以下本次不讲

numpy 是不支持这样操作的

In [6]:
import numpy as np

In [26]:
np.correlate([1, 2, 7, 8], [5, 6])

array([17, 52, 83])

In [24]:
np.correlate([5, 6], [1, 2, 7, 8])

array([83, 52, 17])

In [7]:
np.correlate([5, 6], [1, 2, 7, 8], 'valid')

array([83, 52, 17])

In [21]:
np.convolve([5, 6], [1, 2, 7, 8], 'valid')

array([16, 47, 82])

练习

In [19]:
np.convolve([3, 4], [1, 1, 5, 5], 'valid')

array([ 7, 19, 35])

In [17]:
np.correlate([3, 4], [1, 1, 5, 5])

array([35, 23,  7])

TODO 已经在 http://discuss.gluon.ai/ 上提问

In [27]:
256 * 256 * 3 * 1000

196608000

In [28]:
256 * 256 * 3 /1000000

0.196608